<img src="ost_logo.png" width="240" height="240" align="right"/>
<div style="text-align: left"> <b> Machine Learning </b> <br> MSE FTP MachLe <br> 
<a href="mailto:christoph.wuersch@ost.ch"> Christoph Würsch </a> </div>

# Lab13: A3 Q-Learning in the Grid-World
(`Lab13_A3_QLearning_GridBoard_TEMPLATE.ipynb`)

[Adapted from Jeremy Zhang:](https://towardsdatascience.com/implement-grid-world-with-q-learning-51151747b455)

Whereas $V(s)$ is a mapping from state $s$ to estimated value of that state, the $Q$ function — $Q(s, a)$ is only one component different from $V$ function. Instead of thinking that you receive a value when you are in a specific state $s$, think one step forward, you are in a state $s$ and by taking a specific action $a$ you receive a corresponding value. 

The result of grid world using value iteration was that we get a estimated value $V(s)$ for each state, but to have our policy $\pi(s, a)$, which is a mapping from state $s$ to action $a$, we need to go one step further by choosing the action that could get into the maximum value of next state. However, in Q-function, state and action are paired in the first place, which means when one has the optimal Q-function, he has the optimal action of that state.

Besides the Q-function, we are going to add more fun to our game:
- The agent action is **non-deterministic**, i.e. there is a transition probability $P_{ss'}$ from one state $s$ to another state $s'$
- Reward decays with ratio $\gamma$

Non-deterministic means that the agent will not be able to go where it intends to go. When it takes an action, it will has a probability to crash in a different action.




<img src="board2.png" alt="drawing" width="600"/>

---
When the agent moving up, it has **0.8** probability moving up and **0.1** probability of going left or right. (non-deterministic)



## Q-learning algorithm


One of the early breakthroughs in reinforcement learning was the development of an
off-policy TD control algorithm known as Q-learning (Watkins, 1989), defined by

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t)+ \alpha \cdot \left[ R_{t+1}+ \gamma \max_a Q(s_{t+1},a)-Q(s_t,a_t)\right]$$

- In this case, the learned action-value function, $Q$, directly approximates $Q^*$, the optimal action-value function, *independent of the policy being followed*. This dramatically simplifies the analysis of the algorithm and enabled early convergence proofs. The policy still has an effect in that it determines which state–action pairs are visited and updated. The policy can directly be derived from $Q$, e.g. using an $\varepsilon$-greedy approach.
- However, all that is required for correct convergence is that all pairs continue to be updated. Under this assumption and a variant of the usual stochastic approximation conditions on the sequence of step-size parameters, $Q$ has been shown to converge with probability $1$ to $Q^*$. The Q-learning algorithm is shown below in procedural form.



The *Q-learning-algorithm* is intrinsically **off-policy**, since it does not draw samples from a given policy $\pi$.

1. **Perform an action** $a$ in a state $s$ to end up in $s'$ with reward $r$, i.e. consider the tuple $(s,a,s',r)$
2. **Compute the intermediate Q-value:**
$$ \hat{Q}(s,a)=R(s,a,s')+\gamma \max_{a'} Q_k(s',a')$$

3. Incorporate that new evidence into the existing value using a **running average**: ($\alpha$ is the learning rate)
$$Q_{k+1}(s,a)=(1-\alpha)\cdot Q_k(s,a)+\alpha\cdot \hat{Q}(s,a)$$

This can be rewritten in a **gradient-descent update rule** fashion as:

$$ Q_{k+1}(s,a) = Q_k(s,a)+\alpha \cdot \left( \hat{Q}(s,a)-Q_k(s,a) \right)$$


In [4]:
import numpy as np

In [5]:
BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3)
LOSE_STATE = (1, 3)
START = (2, 0)
DETERMINISTIC = False

In [6]:
class Environment:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC
        
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0
    
    def isEndFunc(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    def _chooseActionProb(self, action):
        if action == "up":
            return np.random.choice(["up", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "down":
            return np.random.choice(["down", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "left":
            return np.random.choice(["left", "up", "down"], p=[0.8, 0.1, 0.1])
        if action == "right":
            return np.random.choice(["right", "up", "down"], p=[0.8, 0.1, 0.1])
        
    def nxtPosition(self, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position on board
        """
        if self.determine:
            if action == "up":
                nxtState = (self.state[0]-1, self.state[1])
            elif action == "down":
                nxtState = (self.state[0]+1, self.state[1])
            elif action == "left":
                nxtState = (self.state[0], self.state[1]-1)
            else:
                nxtState = (self.state[0], self.state[1]+1)
            self.determine = False
        else:
            # non-deterministic
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)
                        
        # if next state is legal
        if (nxtState[0] >= 0) and (nxtState[0] <= 2):
            if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                if nxtState != (1, 1):
                    return nxtState
        return self.state
    
    def showBoard(self):
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')    

### Temporal Difference Learning

- Firstly, at each step, an agent takes action $a$ , collecting corresponding reward $r$ , and moves from state $s$ to $s'$. So a whole pair of $(s, a, s',r)$ is considered at each step.
- Secondly, we give an estimation of current $Q$ value, which equals to current reward plus maximum $Q$ value of next state times a decay rate $\gamma$. One thing worth noting is that we set all intermediate reward as 0, so the agent won’t be able to collect any non-zero reward until the end state, either 1 or -1.(This is not compulsory, you can try out other reward and see how the agent acts)
- Lastly, we update the estimation of the current Q value by adding $\alpha$ times a **temporal difference** (which is the difference between new estimation and current value).

### Action
- In terms of action taking, it will still be based on exploration rate.
- When the agent exploits the state, **it will take the action that maximises the Q value according to current estimation of Q-value**.



In [7]:
class Agent:
    
    def __init__(self):
        self.states = []  # record position and action taken at the position
        self.actions = ["up", "down", "left", "right"]
        self.State = Environment()
        self.isEnd = self.State.isEnd
        self.lr = 0.2
        self.exp_rate = 0.3
        self.decay_gamma = 0.9

        # initial Q values
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0  # Q value is a dict of dict  
    
    def chooseAction(self):
        # choose action with most expected value
        mx_nxt_reward = 0
        action = ""
        
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            for a in self.actions:
                current_position = self.State.state
                nxt_reward = self.Q_values[current_position][a]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
            #print("current pos: {}, greedy aciton: {}".format(self.State.state, action))
        return action
    
    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        # update State
        return Environment(state=position)     
    
    def reset(self):
        self.states = []
        self.State = Environment()
        self.isEnd = self.State.isEnd
    
    def play(self, rounds=10):
        i = 0
        while i < rounds:
            # to the end of game back propagate reward
            if self.State.isEnd:
                # back propagate
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                print("Game End Reward", reward)
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr*(self.decay_gamma*reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                # append trace
                self.states.append([(self.State.state), action])
                print("current position {} action {}".format(self.State.state, action))
                # by taking the action, it reaches the next state
                self.State = self.takeAction(action)
                # mark is end
                self.State.isEndFunc()
                print("nxt state", self.State.state)
                print("---------------------")
                self.isEnd = self.State.isEnd

In [8]:
ag = Agent()

In [9]:
ag.Q_values

{(0, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (0, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (2, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}}

In [10]:
ag.play(50)

current position (2, 0) action down
nxt state (2, 0)
---------------------
current position (2, 0) action up
nxt state (1, 0)
---------------------
current position (1, 0) action right
nxt state (2, 0)
---------------------
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action right
nxt state (2, 2)
---------------------
current position (2, 2) action up
nxt state (1, 2)
---------------------
current position (1, 2) action up
nxt state (0, 2)
---------------------
current position (0, 2) action up
nxt state (0, 2)
---------------------
current position (0, 2) action left
nxt state (0, 1)
---------------------
current position (0, 1) action right
nxt state (0, 1)
---------------------
current position (0, 1) action down
nxt state (0, 1)
---------------------
current position (0, 1) action right
nxt state (0, 2)
---------------------
current position (0, 2) action up
nxt state (0, 2)
---------------------
current position (0, 2) action

nxt state (0, 1)
---------------------
current position (0, 1) action right
nxt state (0, 2)
---------------------
current position (0, 2) action right
nxt state (0, 3)
---------------------
Game End Reward 1
current position (2, 0) action up
nxt state (1, 0)
---------------------
current position (1, 0) action up
nxt state (0, 0)
---------------------
current position (0, 0) action up
nxt state (0, 0)
---------------------
current position (0, 0) action right
nxt state (0, 1)
---------------------
current position (0, 1) action down
nxt state (0, 1)
---------------------
current position (0, 1) action right
nxt state (0, 2)
---------------------
current position (0, 2) action down
nxt state (1, 2)
---------------------
current position (1, 2) action up
nxt state (0, 2)
---------------------
current position (0, 2) action up
nxt state (0, 2)
---------------------
current position (0, 2) action right
nxt state (0, 3)
---------------------
Game End Reward 1
current position (2, 0) action

In [11]:
ag.Q_values

{(0, 0): {'up': 0.277, 'down': 0.072, 'left': 0.207, 'right': 0.401},
 (0, 1): {'up': 0.298, 'down': 0.312, 'left': 0.182, 'right': 0.545},
 (0, 2): {'up': 0.355, 'down': 0.253, 'left': 0.034, 'right': 0.718},
 (0, 3): {'up': 1, 'down': 1, 'left': 1, 'right': 1},
 (1, 0): {'up': 0.248, 'down': 0.126, 'left': 0.058, 'right': 0.148},
 (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 (1, 2): {'up': -0.126, 'down': 0.001, 'left': 0.156, 'right': -0.087},
 (1, 3): {'up': -1, 'down': -1, 'left': -1, 'right': -1},
 (2, 0): {'up': 0.244, 'down': 0.01, 'left': 0.046, 'right': 0.057},
 (2, 1): {'up': 0.075, 'down': -0.001, 'left': 0.092, 'right': -0.006},
 (2, 2): {'up': -0.008, 'down': 0, 'left': 0.009, 'right': -0.045},
 (2, 3): {'up': 0, 'down': 0, 'left': -0.005, 'right': -0.136}}